# ERA5-Land Portugal heat/need exposure indices (2010–2012) — GRID exports (Mollweide)

This notebook computes four indices from **ERA5-Land HOURLY** `skin_temperature` over **Portugal (mainland + Azores + Madeira)**:

- **hdd_18**: count of days where daily mean skin temp < 18°C  
- **cdd_25**: count of days where daily mean skin temp > 25°C  
- **extreme_heat**: mean of the 6th–10th hottest *daily mean* temperatures  
- **extreme_cold**: mean of the 6th–10th coldest *daily mean* temperatures  

It exports each index as:
1) a **single-band GeoTIFF** reprojected to **Mollweide (ESRI:54009)** to Google Drive  
2) a **manifest** (GeoJSON; valid JSON) saved alongside the TIFF in the same Drive folder

> Note: Exports run as Earth Engine tasks. After running the export cell, monitor progress in the EE Tasks UI (or via the status cell).


In [ ]:
# (Colab) Install Earth Engine Python API if needed
!pip -q install earthengine-api geemap

In [ ]:
import datetime
import ee

# Authenticate & initialize (Colab will prompt you)
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

print("EE initialized:", ee.String('ok').getInfo())

## Parameters

In [ ]:
# ---- User parameters ----
DRIVE_FOLDER = "ERA5L_PT_Mollweide_2010_2012"   # Drive folder name to create/use
TARGET_RES_M = 10000                           # Export resolution in meters (Mollweide)
MAX_PIXELS = 1e13

START_YEAR = 2010
END_YEAR = 2012  # inclusive

ERA5_COLLECTION = "ECMWF/ERA5_LAND/HOURLY"
TEMP_BAND = "skin_temperature"                 # Kelvin

MOLL_CRS = "ESRI:54009"                        # Mollweide
USE_BILINEAR = True                            # True for smoother continuous fields


## Portugal region (mainland + Azores + Madeira)

In [ ]:
# Portugal boundary from a built-in dataset (includes the Atlantic islands)
gaul0 = ee.FeatureCollection("FAO/GAUL/2015/level0")
pt_geom = gaul0.filter(ee.Filter.eq("ADM0_NAME", "Portugal")).geometry()

# Optional buffer to be safe on coasts; then bounds for a rectangular export region
pt_region = pt_geom.buffer(20000).bounds()

# Quick sanity check: print area in km^2 (approx)
area_km2 = pt_geom.area().divide(1e6)
print("Portugal (GAUL) area ~ km^2:", area_km2.getInfo())

## Build daily mean temperature collection (°C)

In [ ]:
era5 = ee.ImageCollection(ERA5_COLLECTION).select([TEMP_BAND])

def daily_mean_collection(start_date: ee.Date, end_date: ee.Date) -> ee.ImageCollection:
    """Daily mean (°C) from hourly ERA5-Land. end_date is exclusive."""
    n_days = end_date.difference(start_date, "day")
    day_offsets = ee.List.sequence(0, n_days.subtract(1))

    def one_day(d):
        d = ee.Number(d)
        day_start = start_date.advance(d, "day")
        day_end = day_start.advance(1, "day")
        img_k = era5.filterDate(day_start, day_end).mean()
        img_c = img_k.subtract(273.15).rename("tmean_c")
        return img_c.set({
            "system:time_start": day_start.millis(),
            "date_ymd": day_start.format("YYYY-MM-dd")
        })

    return ee.ImageCollection(day_offsets.map(one_day))

start_date = ee.Date.fromYMD(START_YEAR, 1, 1)
end_date = ee.Date.fromYMD(END_YEAR + 1, 1, 1)  # exclusive

daily = daily_mean_collection(start_date, end_date)
print("Daily images:", daily.size().getInfo())

## Compute yearly indices and average across 2010–2012

In [ ]:
def indices_for_year(year: ee.Number) -> ee.Image:
    y = ee.Number(year)
    y_start = ee.Date.fromYMD(y, 1, 1)
    y_end = y_start.advance(1, "year")
    dly = daily.filterDate(y_start, y_end)

    # hdd_18: count of days with daily mean < 18°C
    hdd18 = dly.map(lambda im: im.lt(18).rename("hdd_18_day")).sum().rename("hdd_18")

    # cdd_25: count of days with daily mean > 25°C
    cdd25 = dly.map(lambda im: im.gt(25).rename("cdd_25_day")).sum().rename("cdd_25")

    # Convert daily ImageCollection to array for per-pixel sorting
    arr = dly.select("tmean_c").toArray()           # [time]
    arr_sorted = arr.arraySort()                    # ascending: coldest -> hottest
    n = arr_sorted.arrayLength(0)

    # 6th-10th coldest (indices 5..9)
    cold_slice = arr_sorted.arraySlice(0, 5, 10)
    extreme_cold = cold_slice.arrayReduce(ee.Reducer.mean(), [0]).arrayGet([0]).rename("extreme_cold")

    # 6th-10th hottest: slice (n-10) .. (n-5)
    hot_start = n.subtract(10)
    hot_end = n.subtract(5)
    hot_slice = arr_sorted.arraySlice(0, hot_start, hot_end)
    extreme_heat = hot_slice.arrayReduce(ee.Reducer.mean(), [0]).arrayGet([0]).rename("extreme_heat")

    return ee.Image.cat([hdd18, cdd25, extreme_heat, extreme_cold]).set({"year": y})

years = ee.List.sequence(START_YEAR, END_YEAR)
yearly = ee.ImageCollection(years.map(indices_for_year))

avg_indices = yearly.mean().clip(pt_region)

if USE_BILINEAR:
    avg_indices = avg_indices.resample("bilinear")

# Set a default projection; export will still use the crs/scale parameters explicitly
avg_indices = avg_indices.setDefaultProjection(ee.Projection(MOLL_CRS).atScale(TARGET_RES_M))

print("Bands:", avg_indices.bandNames().getInfo())

## Export each band as GeoTIFF + manifest (GeoJSON) to Google Drive

In [ ]:
generated_utc = datetime.datetime.utcnow().replace(microsecond=0).isoformat() + "Z"

def export_band_with_manifest(image: ee.Image, band_name: str):
    file_base = f"PT_{band_name}_ERA5L_skinT_dailyMean_{START_YEAR}_{END_YEAR}_Mollweide_{TARGET_RES_M}m"
    tif_name = file_base + ".tif"
    manifest_name = file_base + ".manifest.geojson"

    # --- Export GeoTIFF
    task_img = ee.batch.Export.image.toDrive(
        image=image.select([band_name]).toFloat(),
        description=f"EXPORT_{file_base}",
        folder=DRIVE_FOLDER,
        fileNamePrefix=file_base,
        region=pt_region,
        scale=TARGET_RES_M,
        crs=MOLL_CRS,
        maxPixels=MAX_PIXELS,
        fileFormat="GeoTIFF"
    )
    task_img.start()

    # --- Manifest as a single Feature exported as GeoJSON (valid JSON)
    manifest = {
        "folder": DRIVE_FOLDER,
        "generated_utc": generated_utc,
        "files": {
            tif_name: {
                "file": {
                    "name": tif_name,
                    "src_path": f"gee://{ERA5_COLLECTION}:{TEMP_BAND}",
                    "dst_path": f"gdrive://{DRIVE_FOLDER}/{tif_name}"
                },
                "general": {
                    "action": "computed_and_reprojected",
                    "src_crs": "EPSG:4326 (native lon/lat grid in GEE for ERA5-Land)",
                    "dst_crs": MOLL_CRS,
                    "reprojected": True,
                    "copied_as_is": False,
                    "resampling": "bilinear" if USE_BILINEAR else "nearest/implicit",
                    "target_resolution_m": float(TARGET_RES_M),
                    "target_crs": MOLL_CRS,
                    "region": "Portugal (incl. Azores + Madeira) from FAO/GAUL/2015/level0"
                },
                "specific": {
                    "dataset": ERA5_COLLECTION,
                    "temperature_band": TEMP_BAND,
                    "daily_stat": "mean(hourly) -> daily mean °C",
                    "period_years": [START_YEAR, END_YEAR],
                    "index_definition": band_name,
                    "thresholds_c": {"hdd_18": 18, "cdd_25": 25}
                }
            }
        }
    }

    fc = ee.FeatureCollection([ee.Feature(None, {"manifest": ee.Dictionary(manifest)})])

    task_manifest = ee.batch.Export.table.toDrive(
        collection=fc,
        description=f"EXPORT_MANIFEST_{file_base}",
        folder=DRIVE_FOLDER,
        fileNamePrefix=file_base + ".manifest",
        fileFormat="GeoJSON"
    )
    task_manifest.start()

    return task_img, task_manifest

bands = ["hdd_18", "cdd_25", "extreme_heat", "extreme_cold"]
tasks = {}

for b in bands:
    t_img, t_mani = export_band_with_manifest(avg_indices, b)
    tasks[b] = {"image": t_img, "manifest": t_mani}
    print(f"Started tasks for {b}: image_id={t_img.id}, manifest_id={t_mani.id}")

print("\nExports started. Check Earth Engine Tasks panel (preferred) or run the next cell to poll statuses.")

## (Optional) Poll task statuses

In [ ]:
import time

def show_status():
    for b, tt in tasks.items():
        si = tt["image"].status()
        sm = tt["manifest"].status()
        print(f"{b:13s} | IMG: {si.get('state'):10s} | MANI: {sm.get('state'):10s}")

# Poll a few times (adjust as you like)
for _ in range(5):
    show_status()
    time.sleep(10)
